In [1]:
import numpy as np
import copy as cp
import pickle as pkl
import os.path

exec(open("./diagnostics_header.py").read())
import diagnostics_ops as diops
import diagnostics_vars as divars
import diagnostics_vis as divis

In [2]:
Porb = 2.*np.pi
tstart_steady = {5.:(7.5*Porb), 10.:(34.*Porb)} # sim.u.

# time limits of accretion states
#accretion_state_times = {10.:{'A':[15.5, 18.], 'B':[18., 20.5], 'C':[20.5, 23.5], 'D':[23.5,27.], 'E':[27.,29.]}, 5.:{}} # Porb
accretion_state_times = {10.:{'A':[34.,39.], 'B':[39., 45.6]}, 5.:{}} # Porb

# average disk height to average over for equatorial and radial profile s
# [as theta in units of 1/mach_no ~ thermal scale height]
Havg = 0.5 # just the equator zone (zone 0)

In [3]:
# check which machine we're running on
import socket
machine = socket.gethostname()

# list of tasks to perform in the run
tasks = {}; tasks['1D'] = []; tasks['2D'] = []; tasks['3D'] = []

# parse input
import sys
if machine == 'ppjanka-razer':
    if False:
        supfolder = 'M5/R4_32_dfloor1e-6_vfloorRho1e-5'
        mach_no = 5
    else:
        supfolder = 'M10_noInfl/R5_4b_lowDfloor/R5_dfloor1e-6_vfloorRho1e-5'
        mach_no = 10
else:
    supfolder = sys.argv[1]
    i = 2
    while i < len(sys.argv):
        if sys.argv[i] == '-2D':
            tasks['2D'].append(sys.argv[i+1])
            i += 2
        elif sys.argv[i] == '-1D':
            tasks['1D'].append(sys.argv[i+1])
            i += 2
        elif sys.argv[i] == '-3D':
            tasks['3D'].append(sys.argv[i+1])
            i += 2
            
# default: perform all tasks, WARNING: this can take a long time
if len(tasks['1D']) == 0 and len(tasks['2D']) == 0 and len(tasks['3D']) == 0:
     # 2D profile tasks
    tasks['2D'] = ['alphaR', 'alphaM', 'rho', 'beta', 'B2_over_B1', 'B3_over_B1', 'B2_over_B3', 'alphaRslices', 'vr', 'vrSlices', 'vel3Slices', 'rhoSlices', 'csoundSlices', 'butterfly', 'butterfly_noAvg', 'butterfly2', 'patternSpeed']
    # 1D profile tasks
    tasks['1D'] = ['radial', 'vertical'] # 'poloidal'
    tasks['3D'] = ['rho', 'csound', 'bfield', 'vertical']

In [4]:
if machine[:7] == 'perseus':
    machine = 'PERSEUS'
    data_pathstem = '/home/ppjanka/tigress_pp/THESIS/%s/' % supfolder
elif machine[:11] == 'tigressdata':
    machine = 'TIGRESSDATA'
    data_pathstem = '/home/ppjanka/tigress_pp/THESIS/%s/' % supfolder
elif machine == 'ppjanka-razer':
    machine = 'RAZER'
    data_pathstem = '/DATA/Dropbox/LOOTRPV/astro_projects/2017_globAcc1/athena/bin-mhd/%s/' % supfolder

if machine != 'RAZER':
    mach_no = float(data_pathstem.split('/')[-2].split('M')[1].split('_')[0])
    
print('Running on %s. Mach number: %f.' % (machine, mach_no))

data_folders = ['R4','R5']

# ----------
import glob

athdf_files = []
for data_folder in data_folders:
    athdf_files += glob.glob(data_pathstem + data_folder + '/*out2*.athdf')
athdf_files = sorted(athdf_files)

def state2title (acc_state):
    if acc_state == None:
        return 'steady'
    else:
        return 'state%s' % acc_state

Running on RAZER. Mach number: 10.000000.


In [ ]:
def profiles_2D (vars_obj, acc_state=None):

    # these will hold the final results
    ops_obj_eq = None 
    ops_obj_pol = None

    navg_eq = 0; navg_pol = 0
    for athdf_file in athdf_files:
        print('Processing %s.. ' % athdf_file.split('/')[-1], end='', flush=True)

        if (acc_state == None and diops.get_time(athdf_file) > tstart_steady[mach_no]) or (acc_state != None and diops.get_time(athdf_file) > accretion_state_times[mach_no][acc_state][0]*Porb and diops.get_time(athdf_file) < accretion_state_times[mach_no][acc_state][1]*Porb):
            # equatorial profile
            buff_obj_eq = diops.Profile_PhiR(vars_obj, \
                    x2min=0.5*np.pi-Havg/mach_no, \
                    x2max=0.5*np.pi+Havg/mach_no)
            buff_obj_eq.read(athdf_file)
            if ops_obj_eq == None:
                ops_obj_eq = cp.deepcopy(buff_obj_eq)
                ops_obj_eq.time = tstart_steady[mach_no]
                navg_eq += 1
            else:
                if (buff_obj_eq.phi == ops_obj_eq.phi).all() and (buff_obj_eq.r == ops_obj_eq.r).all():
                    ops_obj_eq.val += buff_obj_eq.val
                    navg_eq += 1
                else:
                    print('Eq. mesh doesn\'t match for %s. Ignored.' % athdf_file.split('/')[-1])

            # poloidal profile
            buff_obj_pol = diops.Profile_ThetaR(vars_obj)
            buff_obj_pol.read(athdf_file)
            if ops_obj_pol == None:
                ops_obj_pol = cp.deepcopy(buff_obj_pol)
                ops_obj_pol.time = tstart_steady[mach_no]
                navg_pol += 1
            else:
                if (buff_obj_pol.theta == ops_obj_pol.theta).all() and (buff_obj_pol.r == ops_obj_pol.r).all():
                    ops_obj_pol.val += buff_obj_pol.val
                    navg_pol += 1
                else:
                    print('Pol. mesh doesn\'t match for %s. Ignored.' % athdf_file.split('/')[-1])
        else:
            print('frame outside the requested time range. ', end='', flush=True)
        print('done.', flush=True)

    ops_obj_eq.val /= navg_eq
    ops_obj_pol.val /= navg_pol

    print('\nDONE.\n', flush=True)
    
    return ops_obj_eq, ops_obj_pol

In [ ]:
if 'alphaR' in tasks['2D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_AlphaReynolds.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        # alpha_Reynolds averaged throughout the steady state
        vars_obj = divars.AlphaReynolds()
        ops_obj_eq, ops_obj_pol = profiles_2D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump((ops_obj_eq, ops_obj_pol), f)
        
        del vars_obj, ops_obj_eq, ops_obj_pol

In [ ]:
if 'alphaM' in tasks['2D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_AlphaMaxwell.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        # alpha_Maxwell averaged throughout the steady state
        vars_obj = divars.AlphaMaxwell()
        ops_obj_eq, ops_obj_pol = profiles_2D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump((ops_obj_eq, ops_obj_pol), f)

        del vars_obj, ops_obj_eq, ops_obj_pol

In [ ]:
if 'rho' in tasks['2D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_rho.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        # rho averaged throughout the steady state
        vars_obj = divars.Default('rho')
        ops_obj_eq, ops_obj_pol = profiles_2D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump((ops_obj_eq, ops_obj_pol), f)

        del vars_obj, ops_obj_eq, ops_obj_pol

In [ ]:
if 'vr' in tasks['2D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_vr.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        # rho averaged throughout the steady state
        vars_obj = divars.Default('vel1')
        ops_obj_eq, ops_obj_pol = profiles_2D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump((ops_obj_eq, ops_obj_pol), f)

        del vars_obj, ops_obj_eq, ops_obj_pol

In [ ]:
if 'beta' in tasks['2D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_PlasmaBeta.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        # plasma beta averaged throughout the steady state
        vars_obj = divars.PlasmaBeta()
        ops_obj_eq, ops_obj_pol = profiles_2D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump((ops_obj_eq, ops_obj_pol), f)

        del vars_obj, ops_obj_eq, ops_obj_pol

In [ ]:
if 'B2_over_B1' in tasks['2D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_B2_over_B1.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        vars_obj = divars.B2_over_B1()
        ops_obj_eq, ops_obj_pol = profiles_2D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump((ops_obj_eq, ops_obj_pol), f)

        del vars_obj, ops_obj_eq, ops_obj_pol

In [ ]:
if 'B3_over_B1' in tasks['2D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_B3_over_B1.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        vars_obj = divars.B3_over_B1()
        ops_obj_eq, ops_obj_pol = profiles_2D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump((ops_obj_eq, ops_obj_pol), f)

        del vars_obj, ops_obj_eq, ops_obj_pol

In [ ]:
if 'B2_over_B3' in tasks['2D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_B2_over_B3.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        vars_obj = divars.B2_over_B3()
        ops_obj_eq, ops_obj_pol = profiles_2D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump((ops_obj_eq, ops_obj_pol), f)

        del vars_obj, ops_obj_eq, ops_obj_pol

In [ ]:
def slices_2D (vars_obj, n_poloidal=0, acc_state=None):

    # these will hold the final results
    ops_obj_eq = None
    navg_eq = 0
    if n_poloidal > 0:
        ops_obj_pol = n_poloidal * [None,]
        navg_pol = n_poloidal * [0,]
    else:
        ops_obj_pol = None
        navg_pol = None

    for athdf_file in athdf_files:
        print('Processing %s.. ' % athdf_file.split('/')[-1], end='', flush=True)

        if (acc_state == None and diops.get_time(athdf_file) > tstart_steady[mach_no]) or (acc_state != None and diops.get_time(athdf_file) > accretion_state_times[mach_no][acc_state][0]*Porb and diops.get_time(athdf_file) < accretion_state_times[mach_no][acc_state][1]*Porb):
            
            # equatorial slice
            buff_obj_eq = diops.EquatorialSlice(vars_obj)
            buff_obj_eq.read(athdf_file)
            if ops_obj_eq == None:
                ops_obj_eq = cp.deepcopy(buff_obj_eq)
                ops_obj_eq.time = tstart_steady[mach_no]
                navg_eq += 1
            else:
                if (buff_obj_eq.phi == ops_obj_eq.phi).all() and (buff_obj_eq.r == ops_obj_eq.r).all():
                    ops_obj_eq.val += buff_obj_eq.val
                    navg_eq += 1
                else:
                    print('Eq. mesh doesn\'t match for %s. Ignored.' % athdf_file.split('/')[-1])

            if n_poloidal > 0:
                # poloidal profile
                for i in range(n_poloidal):
                    buff_obj_pol = diops.PoloidalSlice(vars_obj, intersect=(i*2.*np.pi/n_poloidal+1.0e-3))
                    buff_obj_pol.read(athdf_file)
                    if ops_obj_pol[i] == None:
                        ops_obj_pol[i] = cp.deepcopy(buff_obj_pol)
                        ops_obj_pol[i].time = tstart_steady[mach_no]
                        navg_pol[i] += 1
                    else:
                        if (buff_obj_pol.theta == ops_obj_pol[i].theta).all() and (buff_obj_pol.r == ops_obj_pol[i].r).all():
                            ops_obj_pol[i].val += buff_obj_pol.val
                            navg_pol[i] += 1
                        else:
                            print('Pol. mesh doesn\'t match for %s. Ignored.' % athdf_file.split('/')[-1])
                            
        else:
            print('frame outside the requested time range. ', end='', flush=True)
        print('done.', flush=True)

    ops_obj_eq.val /= navg_eq
    for i in range(n_poloidal):
        ops_obj_pol[i].val /= navg_pol[i]

    print('\nDONE.\n', flush=True)
    
    return ops_obj_eq, ops_obj_pol

In [ ]:
if 'alphaRslices' in tasks['2D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_AlphaReynoldsSlices.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        # alpha_Reynolds averaged throughout the steady state
        vars_obj = divars.AlphaReynolds()
        ops_obj_eq, ops_obj_pol = slices_2D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump((ops_obj_eq, ops_obj_pol), f)
        
        del vars_obj, ops_obj_eq, ops_obj_pol

In [ ]:
if 'vrSlices' in tasks['2D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_vrSlices.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        # alpha_Reynolds averaged throughout the steady state
        vars_obj = divars.Default('vel1')
        ops_obj_eq, ops_obj_pol = slices_2D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump((ops_obj_eq, ops_obj_pol), f)
        
        del vars_obj, ops_obj_eq, ops_obj_pol

In [ ]:
if 'vel3Slices' in tasks['2D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_vel3Slices.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        # alpha_Reynolds averaged throughout the steady state
        vars_obj = divars.Default('vel3')
        ops_obj_eq, ops_obj_pol = slices_2D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump((ops_obj_eq, ops_obj_pol), f)
        
        del vars_obj, ops_obj_eq, ops_obj_pol

In [ ]:
if 'rhoSlices' in tasks['2D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_rhoSlices.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        # alpha_Reynolds averaged throughout the steady state
        vars_obj = divars.Default('rho')
        ops_obj_eq, ops_obj_pol = slices_2D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump((ops_obj_eq, ops_obj_pol), f)
        
        del vars_obj, ops_obj_eq, ops_obj_pol

In [ ]:
if 'csoundSlices' in tasks['2D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_csoundSlices.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        # alpha_Reynolds averaged throughout the steady state
        vars_obj = divars.Csound()
        ops_obj_eq, ops_obj_pol = slices_2D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump((ops_obj_eq, ops_obj_pol), f)
        
        del vars_obj, ops_obj_eq, ops_obj_pol

In [ ]:
if 'bfieldSlices' in tasks['2D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_bfieldSlices.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        # alpha_Reynolds averaged throughout the steady state
        vars_obj = divars.Bfield()
        ops_obj_eq, ops_obj_pol = slices_2D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump((ops_obj_eq, ops_obj_pol), f)
        
        del vars_obj, ops_obj_eq, ops_obj_pol

In [ ]:
def fullAvg_3D (vars_obj, acc_state=None):

    # these will hold the final results
    ops_obj = None

    navg = 0
    for athdf_file in athdf_files:
        print('Processing %s.. ' % athdf_file.split('/')[-1], end='', flush=True)

        if (acc_state == None and diops.get_time(athdf_file) > tstart_steady[mach_no]) or (acc_state != None and diops.get_time(athdf_file) > accretion_state_times[mach_no][acc_state][0]*Porb and diops.get_time(athdf_file) < accretion_state_times[mach_no][acc_state][1]*Porb):
            
            buff_obj = diops.Full3D(vars_obj)
            buff_obj.read(athdf_file)
            if ops_obj == None:
                ops_obj = cp.deepcopy(buff_obj)
                ops_obj.time = tstart_steady[mach_no]
                navg += 1
            else:
                ops_obj.val += buff_obj.val
                navg += 1

        else:
            print('frame outside the requested time range. ', end='', flush=True)
        print('done.', flush=True)

    ops_obj.val /= navg

    print('\nDONE.\n', flush=True)
    
    return ops_obj

In [ ]:
if 'rho' in tasks['3D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_rho3D.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        # rho averaged throughout the steady state
        vars_obj = divars.Default('rho')
        ops_obj = fullAvg_3D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump(ops_obj, f)

        del vars_obj, ops_obj

In [ ]:
if 'vel1' in tasks['3D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_vel13D.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        # rho averaged throughout the steady state
        vars_obj = divars.Default('vel1')
        ops_obj = fullAvg_3D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump(ops_obj, f)

        del vars_obj, ops_obj
        
if 'vel2' in tasks['3D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_vel23D.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        # rho averaged throughout the steady state
        vars_obj = divars.Default('vel2')
        ops_obj = fullAvg_3D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump(ops_obj, f)

        del vars_obj, ops_obj
        
if 'vel3' in tasks['3D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_vel33D.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        # rho averaged throughout the steady state
        vars_obj = divars.Default('vel3')
        ops_obj = fullAvg_3D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump(ops_obj, f)

        del vars_obj, ops_obj

In [ ]:
# absolute values of magnetic field components (used for lambda_MRI)
if 'absBcc1' in tasks['3D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_absBcc13D.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        # rho averaged throughout the steady state
        vars_obj = divars.Default('absBcc1')
        ops_obj = fullAvg_3D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump(ops_obj, f)

        del vars_obj, ops_obj
        
if 'absBcc2' in tasks['3D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_absBcc23D.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        # rho averaged throughout the steady state
        vars_obj = divars.Default('absBcc2')
        ops_obj = fullAvg_3D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump(ops_obj, f)

        del vars_obj, ops_obj
        
if 'absBcc3' in tasks['3D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_absBcc33D.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        # rho averaged throughout the steady state
        vars_obj = divars.Default('absBcc3')
        ops_obj = fullAvg_3D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump(ops_obj, f)

        del vars_obj, ops_obj

In [ ]:
if 'csound' in tasks['3D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_csound3D.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        # rho averaged throughout the steady state
        vars_obj = divars.Csound()
        ops_obj = fullAvg_3D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump(ops_obj, f)

        del vars_obj, ops_obj

In [ ]:
if 'bfield' in tasks['3D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_bfield3D.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
        
        # rho averaged throughout the steady state
        vars_obj = divars.Bfield()
        ops_obj = fullAvg_3D(vars_obj, acc_state=acc_state)

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump(ops_obj, f)

        del vars_obj, ops_obj

In [ ]:
def radial_profiles_1D (vars_objs, straddle=None, \
                        x2min=0.5*np.pi-Havg/mach_no, \
                        x2max=0.5*np.pi+Havg/mach_no, \
                        ops_kwargs={}, acc_state=None):

    # these will hold the final results
    ops_objs = {}

    navg = {}
    for athdf_file in athdf_files:
        print('Processing %s.. ' % athdf_file.split('/')[-1], end='', flush=True)

        if (acc_state == None and diops.get_time(athdf_file) > tstart_steady[mach_no]) or (acc_state != None and diops.get_time(athdf_file) > accretion_state_times[mach_no][acc_state][0]*Porb and diops.get_time(athdf_file) < accretion_state_times[mach_no][acc_state][1]*Porb):
            for vars_obj in vars_objs:
                label = vars_obj.label
                if label in ops_kwargs.keys():
                    kwargs = ops_kwargs[label]
                else:
                    kwargs = {}
                if straddle == None:
                    buff_obj = diops.RadialProfile(vars_obj, \
                        x2min=x2min, \
                        x2max=x2max, \
                        **kwargs)
                else:
                    buff_obj = diops.RadialProfile_Straddle(vars_obj, \
                        x2min=straddle[0], \
                        x2max=straddle[1], \
                        **kwargs)
                buff_obj.read(athdf_file)
                if label not in ops_objs.keys():
                    ops_objs[label] = cp.deepcopy(buff_obj)
                    ops_objs[label].time = tstart_steady
                    navg[label] = 1
                else:
                    if (buff_obj.r == ops_objs[label].r).all():
                        ops_objs[label].val += buff_obj.val
                        navg[label] += 1
                    else:
                        print('Mesh doesn\'t match for %s, label %s. Ignored.' % (athdf_file.split('/')[-1], label))
                    
        else:
            print('frame outside the requested time range. ', end='', flush=True)
        print('done.', flush=True)

    for label in ops_objs.keys():
        ops_objs[label].val /= navg[label]

    print('\nDONE.\n', flush=True)
    
    return ops_objs

In [ ]:
if 'radial' in tasks['1D']:
    # read the 1D profiles
    if mach_no == 5.:
        sup_kwargs = [{'x2min':0.5*np.pi-2.5/mach_no, \
                       'x2max':0.5*np.pi+2.5/mach_no}, \
                      {'straddle':(0.5*np.pi-5.0/mach_no, \
                                   0.5*np.pi-2.5/mach_no)}]
    elif mach_no == 10.:
        sup_kwargs = [{'x2min':0.5*np.pi-2./mach_no, \
                       'x2max':0.5*np.pi+2./mach_no}, \
                      {'straddle':(0.5*np.pi-10./mach_no, \
                                   0.5*np.pi-2./mach_no)}]

    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_radial.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue
            
        ops_objs = {}
        ops_objs['sup_kwargs'] = sup_kwargs
            
        for idx in range(len(sup_kwargs)):
            ops_objs[idx] = radial_profiles_1D([divars.Default('rho'),
                               divars.PressMag(), divars.Default('press'),
                               divars.Mdot(),
                               divars.AlphaReynolds(), divars.AlphaMaxwell()],
                               ops_kwargs={'rho':{'type':'surface_density'}, 'Mdot':{'type':'sum'}}, 
                                **(sup_kwargs[idx]), acc_state=acc_state)
            ops_objs[idx]['Ptot'] = cp.deepcopy(ops_objs[idx]['press'])
            ops_objs[idx]['Ptot'].val += ops_objs[idx]['Pmag'].val
            ops_objs[idx]['alpha_tot'] = cp.deepcopy(ops_objs[idx]['alpha_Reynolds'])
            ops_objs[idx]['alpha_tot'].val += ops_objs[idx]['alpha_Maxwell'].val

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump(ops_objs, f)
            
        del ops_objs

In [ ]:
def poloidal_profiles_1D (vars_objs, radii, ops_kwargs={}, acc_state=None):

    # these will hold the final results
    ops_objs = {}

    navg = {}
    for athdf_file in athdf_files:
        print('Processing %s.. ' % athdf_file.split('/')[-1], end='', flush=True)

        if (acc_state == None and diops.get_time(athdf_file) > tstart_steady[mach_no]) or (acc_state != None and diops.get_time(athdf_file) > accretion_state_times[mach_no][acc_state][0]*Porb and diops.get_time(athdf_file) < accretion_state_times[mach_no][acc_state][1]*Porb):
            for vars_obj in vars_objs:
                label = vars_obj.label
                if label in ops_kwargs.keys():
                    kwargs = ops_kwargs[label]
                else:
                    kwargs = {}
                if label not in ops_objs.keys():
                    ops_objs[label] = {}
                    navg[label] = {}
                for radius in radii:
                    buff_obj = {}
                    buff_obj['slice'] = diops.Profile_Theta(vars_obj, \
                            x1min=radius, x1max=radius, \
                            x3min=np.pi, x3max=np.pi, \
                            **kwargs)
                    buff_obj['avg'] = diops.Profile_Theta(vars_obj, \
                            x1min=radius, x1max=radius, \
                            **kwargs)
                    for t in ['slice', 'avg']:
                        buff_obj[t].read(athdf_file)
                    if radius not in ops_objs[label].keys():
                        ops_objs[label][radius] = {}
                        navg[label][radius] = {}
                        for t in ['slice', 'avg']:
                            ops_objs[label][radius][t] = cp.deepcopy(buff_obj[t])
                            ops_objs[label][radius][t].time = tstart_steady[mach_no]
                            navg[label][radius][t] = 1
                    else:
                        for t in ['slice', 'avg']:
                            if (buff_obj[t].theta == ops_objs[label][radius][t].theta).all():
                                ops_objs[label][radius][t].val += buff_obj[t].val
                                navg[label][radius][t] += 1
                            else:
                                print('Mesh doesn\'t match for %s, label %s, radius %f, type %s. Ignored.' % (athdf_file.split('/')[-1], label, radius, t))
                    
        else:
            print('frame outside the requested time range. ', end='', flush=True)
        print('done.', flush=True)

    for label in ops_objs.keys():
        for radius in ops_objs[label].keys():
            for t in ops_objs[label][radius].keys():
                ops_objs[label][radius][t].val /= navg[label][radius][t]

    print('\nDONE.\n', flush=True)
    
    return ops_objs

In [ ]:
if 'poloidal' in tasks['1D']:
    
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        outfile = data_pathstem + ('%s_poloidal.pkl' % state2title(acc_state))
        
        if os.path.isfile(outfile): continue

        # read the 1D profiles
        ops_objs_poloidal = poloidal_profiles_1D([divars.Default('rho'),
                           divars.PressMag(), divars.Default('press'),
                           divars.Mdot(),
                           divars.AlphaReynolds(),
                           divars.AlphaMaxwell(),
                           divars.StressMaxwell(),
                           divars.StressReynolds()],
                    radii=[0.06, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
                    ops_kwargs={'Mdot':{'type':'sum'}}, acc_state=acc_state)

        ops_objs_poloidal['Ptot'] = {}
        ops_objs_poloidal['alpha_tot'] = {}
        for radius in ops_objs_poloidal['rho'].keys():
            ops_objs_poloidal['Ptot'][radius] = {}
            ops_objs_poloidal['alpha_tot'][radius] = {}
            for t in ops_objs_poloidal['rho'][radius].keys():
                ops_objs_poloidal['Ptot'][radius][t] = cp.deepcopy(ops_objs_poloidal['press'][radius][t])
                ops_objs_poloidal['Ptot'][radius][t].val += ops_objs_poloidal['Pmag'][radius][t].val
                ops_objs_poloidal['alpha_tot'][radius][t] = cp.deepcopy(ops_objs_poloidal['alpha_Reynolds'][radius][t])
                ops_objs_poloidal['alpha_tot'][radius][t].val += ops_objs_poloidal['alpha_Maxwell'][radius][t].val

        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump(ops_objs_poloidal, f)
            
        del ops_objs_poloidal

In [ ]:
def vertical_profiles_1D (vars_objs, radii, x2min=0.1, x2max=(np.pi-0.1), ops_kwargs={}, acc_states=[None,]):

    # these will hold the final results, initialize
    ops_objs = {}
    navg = {}
    for acc_state in acc_states:
        ops_objs[acc_state] = {}
        navg[acc_state] = {}
        for vars_obj in vars_objs:
            label = vars_obj.label
            if label not in ops_objs[acc_state].keys():
                ops_objs[acc_state][label] = {}
                navg[acc_state][label] = {}
                for t in ['slice', 'avg']:
                    navg[acc_state][label][t] = {}
                    for radius in radii:
                        navg[acc_state][label][t][radius] = 1
            if label not in ops_kwargs.keys():
                ops_kwargs[label] = {}
            
    # ------------------------------------
    
    for athdf_file in athdf_files:
        
        print('Processing %s.. ' % athdf_file.split('/')[-1], flush=True)
        
        # ignore if not in any accretion state
        calculate = False
        for acc_state in acc_states:
            if (acc_state == None and diops.get_time(athdf_file) > tstart_steady[mach_no]) or (acc_state != None and diops.get_time(athdf_file) > accretion_state_times[mach_no][acc_state][0]*Porb and diops.get_time(athdf_file) < accretion_state_times[mach_no][acc_state][1]*Porb):
                calculate = True
                break
        if not calculate:
            print('      > frame outside any of the requested time ranges. ', flush=True)
            continue

        # calculate the profiles
        for vars_obj in vars_objs:

            label = vars_obj.label
            print(' - %s.. ' % label, flush=True)

            kwargs = ops_kwargs[label]

            buff_obj = {}
            buff_obj['slice'] = \
                diops.Profile_Cyl_Vertical(vars_obj, radii, \
                    x2min=x2min, x2max=x2max,\
                    x3min=np.pi, x3max=np.pi,\
                    nproc=nproc, **kwargs)
            buff_obj['avg'] = \
                diops.Profile_Cyl_Vertical(vars_obj, radii,\
                    x2min=x2min, x2max=x2max,\
                    nproc=nproc, **kwargs)

            for t in ['slice', 'avg']:
                buff_obj[t].read(athdf_file)
        
            for acc_state in acc_states:

                print('   - state %s' % acc_state, flush=True)

                if (acc_state == None and diops.get_time(athdf_file) > tstart_steady[mach_no]) or (acc_state != None and diops.get_time(athdf_file) > accretion_state_times[mach_no][acc_state][0]*Porb and diops.get_time(athdf_file) < accretion_state_times[mach_no][acc_state][1]*Porb):

                    if len(ops_objs[acc_state][label].keys()) == 0:
                        ops_objs[acc_state][label] = cp.deepcopy(buff_obj)
                    else:
                        for t in ['slice', 'avg']:
                            for radius in radii:
                                ops_objs[acc_state][label][t].val[radius] += buff_obj[t].val[radius]
                                navg[acc_state][label][t][radius] += 1
  
                    print('      done.', flush=True)

                else:
                    print('      > frame outside the requested time range. ', flush=True)
                
            # clean up
            del buff_obj
        
    for acc_state in acc_states:

        for label in ops_objs[acc_state].keys():
            for t in ops_objs[acc_state][label].keys():
                for radius in ops_objs[acc_state][label][t].val.keys():
                    ops_objs[acc_state][label][t].val[radius] /= navg[acc_state][label][t][radius]

    print('\nDONE.\n', flush=True)

    return ops_objs

In [ ]:
if 'vertical' in tasks['1D']:
    
    acc_states = []
    outfiles = {}
    for acc_state in list(accretion_state_times[mach_no].keys()) + [None,]:
        outfiles[acc_state] = data_pathstem + ('%s_vertical.pkl' % state2title(acc_state))
        if not os.path.isfile(outfiles[acc_state]):
            acc_states.append(acc_state)
            
    if mach_no == 5.:
        x2min = 0.5*np.pi - 5./mach_no
        x2max = 0.5*np.pi + 5./mach_no
    elif mach_no == 10.:
        x2min = 0.5*np.pi - 10./mach_no
        x2max = 0.5*np.pi + 10./mach_no

    # read the 1D profiles
    ops_objs_vertical = vertical_profiles_1D([divars.Default('rho'),
                           divars.PressMag(), divars.Default('press'),
                           divars.Mdot(vertical=True),
                           divars.AlphaReynolds(),
                           divars.AlphaMaxwell(),
                           divars.StressMaxwell(),
                           divars.StressReynolds()],
                    radii=[0.06, 0.1, 0.15, 0.2, 0.25],
                    x2min=x2min, x2max=x2max,
                    ops_kwargs={'Mdot':{'type':'sum'}},
                    acc_states=acc_states)

    for acc_state in acc_states:
    
        ops_objs_vertical[acc_state]['Ptot'] = {}
        ops_objs_vertical[acc_state]['alpha_tot'] = {}

        for t in ops_objs_vertical[acc_state]['rho'].keys():
            ops_objs_vertical[acc_state]['Ptot'][t] = cp.deepcopy(ops_objs_vertical[acc_state]['press'][t])
            ops_objs_vertical[acc_state]['alpha_tot'][t] = cp.deepcopy(ops_objs_vertical[acc_state]['alpha_Reynolds'][t])
            for radius in ops_objs_vertical[acc_state]['rho'][t].radii:
                ops_objs_vertical[acc_state]['Ptot'][t].val[radius] += ops_objs_vertical[acc_state]['Pmag'][t].val[radius]
                ops_objs_vertical[acc_state]['alpha_tot'][t].val[radius] += ops_objs_vertical[acc_state]['alpha_Maxwell'][t].val[radius]

        # save for reuse
        with open(outfiles[acc_state], 'wb') as f:
            pkl.dump(ops_objs_vertical[acc_state], f)
            
        del ops_objs_vertical

In [ ]:
if 'vertical' in tasks['3D']:
        
    for acc_state in [None,]+list(accretion_state_times[mach_no].keys()):
        
        print('ACCRETION STATE: %s' % state2title(acc_state), flush=True)
    
        for vars_obj in [divars.Default('rho'),
                           divars.PressMag(), divars.Default('press'),
                           divars.Mdot(vertical=True),
                           divars.AlphaReynolds(),
                           divars.AlphaMaxwell(),
                           divars.StressMaxwell(),
                           divars.StressReynolds()]:
            
            print('Processing %s...' % vars_obj.label, flush=True)

            outfile = data_pathstem + ('%s_%s3D.pkl' % (state2title(acc_state), vars_obj.label))

            if os.path.isfile(outfile):
                print(' -- already exists', flush=True)
                continue

            # averaged throughout the steady state
            ops_obj = fullAvg_3D(vars_obj, acc_state=acc_state)

            # save for reuse
            with open(outfile, 'wb') as f:
                pkl.dump(ops_obj, f)

            del vars_obj, ops_obj
            print(' -- done.')

In [ ]:
def vertical_profiles_1D_from3D (vars_objs, radii, ops_kwargs={}, acc_states=[None,]):

    # these will hold the final results, initialize
    ops_objs = {}
    for acc_state in acc_states:
        ops_objs[acc_state] = {}
        for vars_obj in vars_objs:
            label = vars_obj.label
            if label not in ops_objs[acc_state].keys():
                ops_objs[acc_state][label] = {}
            if label not in ops_kwargs.keys():
                ops_kwargs[label] = {}
            
    # ------------------------------------
    
    for vars_obj in vars_objs:

        label = vars_obj.label
        print(' - %s.. ' % label, flush=True)
        kwargs = ops_kwargs[label]
        
        for acc_state in acc_states:

            print('   - state %s' % acc_state, flush=True)
            
            outfile = data_pathstem + ('%s_%s3D.pkl' % (state2title(acc_state), vars_obj.label))

            if not os.path.isfile(outfile):
                print(' -- waiting for file', flush=True)
                continue

            # load 3D file
            with open(outfile, 'rb') as f:
                ops_obj_3D = pkl.load(f)

            buff_obj = {}
            buff_obj['slice'] = \
                diops.Profile_Cyl_Vertical(vars_obj, radii, \
                    x2min=x2min, x2max=x2max,\
                    x3min=np.pi, x3max=np.pi,\
                    nproc=nproc, **kwargs)
            buff_obj['avg'] = \
                diops.Profile_Cyl_Vertical(vars_obj, radii,\
                    x2min=x2min, x2max=x2max,\
                    x3min=0., x3max=(2.*np.pi),\
                    nproc=nproc, **kwargs)

            for t in ['slice', 'avg']:
                buff_obj[t].read_from_3D(ops_obj_3D)

            ops_objs[acc_state][label] = cp.deepcopy(buff_obj)
    
    return ops_objs

In [ ]:
force_calc = True
if 'vertical' in tasks['3D']:
    
    acc_states = []
    outfiles = {}
    for acc_state in list(accretion_state_times[mach_no].keys()) + [None,]:
        outfiles[acc_state] = data_pathstem + ('%s_vertical.pkl' % state2title(acc_state))
        if not os.path.isfile(outfiles[acc_state]) or force_calc:
            acc_states.append(acc_state)
            
    if mach_no == 5.:
        x2min = 0.5*np.pi - 5./mach_no
        x2max = 0.5*np.pi + 5./mach_no
    elif mach_no == 10.:
        x2min = 0.5*np.pi - 10./mach_no
        x2max = 0.5*np.pi + 10./mach_no

    # read the 1D profiles
    ops_objs_vertical = vertical_profiles_1D_from3D([divars.Default('rho'),
                           divars.PressMag(), divars.Default('press'),
                           divars.Mdot(vertical=True),
                           divars.AlphaReynolds(), # incorrect avg !
                           divars.AlphaMaxwell(), # incorrect avg !
                           divars.StressMaxwell(),
                           divars.StressReynolds()],
                    radii=[0.08, 0.11, 0.15, 0.2, 0.25],
                    ops_kwargs={'Mdot':{'type':'sum'}},
                    acc_states=acc_states)

    for acc_state in acc_states:
    
        ops_objs_vertical[acc_state]['Ptot'] = {}
        ops_objs_vertical[acc_state]['alpha_tot'] = {}

        for t in ops_objs_vertical[acc_state]['rho'].keys():
            ops_objs_vertical[acc_state]['Ptot'][t] = cp.deepcopy(ops_objs_vertical[acc_state]['press'][t])
            ops_objs_vertical[acc_state]['alpha_tot'][t] = cp.deepcopy(ops_objs_vertical[acc_state]['alpha_Reynolds'][t])
            for radius in ops_objs_vertical[acc_state]['rho'][t].radii:
                ops_objs_vertical[acc_state]['Ptot'][t].val[radius] += ops_objs_vertical[acc_state]['Pmag'][t].val[radius]
                ops_objs_vertical[acc_state]['alpha_tot'][t].val[radius] += ops_objs_vertical[acc_state]['alpha_Maxwell'][t].val[radius]

        # save for reuse
        with open(outfiles[acc_state], 'wb') as f:
            pkl.dump(ops_objs_vertical[acc_state], f)

    del ops_objs_vertical

In [ ]:
if 'butterfly' in tasks['2D']:
    
    z_over_H_max = 6.0
    z_over_r_max = z_over_H_max / mach_no
    
    dx2 = np.arctan(z_over_r_max) + 0.1
    
    for direction in ['Bcc3', 'Bcc2', 'Bcc1']:
        
        print('Processing direction %s' % direction)
        
        outfile = data_pathstem + ('butterfly_%s.pkl' % direction)
        
        vars_obj = divars.Default(direction)
        
        # reload if exists
        if os.path.isfile(outfile):
            with open(outfile, 'rb') as f:
                ops_obj = pkl.load(f)
        else:
            ops_obj = diops.ButterflyDiagram(vars_obj, \
                radii=[0.08, 0.11, 0.15, 0.2, 0.25], \
                x2min=(0.5*np.pi - dx2), x2max=(0.5*np.pi + dx2), \
                nz=1024)
        
        ops_obj.read(athdf_files, verbose=True, \
                temp_save_path=outfile)
        
        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump(ops_obj, f)

In [ ]:
if 'butterfly_noAvg' in tasks['2D']:
    
    z_over_H_max = 6.0
    z_over_r_max = z_over_H_max / mach_no
    
    dx2 = np.arctan(z_over_r_max) + 0.1
    
    for direction in ['Bcc3', 'Bcc2', 'Bcc1']:
        
        print('Processing direction %s' % direction)
        
        outfile = data_pathstem + ('butterfly_noAvg_%s.pkl' % direction)
        
        vars_obj = divars.Default(direction)
        
        # reload if exists
        if os.path.isfile(outfile):
            with open(outfile, 'rb') as f:
                ops_obj = pkl.load(f)
        else:
            ops_obj = diops.ButterflyDiagram(vars_obj, \
                radii=[0.08, 0.11, 0.15, 0.2, 0.25], \
                x2min=(0.5*np.pi - dx2), x2max=(0.5*np.pi + dx2), \
                x3min=np.pi, x3max=np.pi, \
                nz=1024)
        
        ops_obj.read(athdf_files, verbose=True, \
                temp_save_path=outfile)
        
        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump(ops_obj, f)

In [ ]:
if 'butterfly2' in tasks['2D']:
    
    z_over_H_max = 6.0
    z_over_r_max = z_over_H_max / mach_no
    
    dx2 = np.arctan(z_over_r_max) + 0.1
    
    for direction in ['KinHelicityRMS','vel3rms']:# ['vel1rms', 'vel2rms', 'vel3rms', 'EMF1','EMF2','EMF3', 'Turb_Ekin']:
        
        print('Processing direction %s' % direction)
        
        outfile = data_pathstem + ('butterfly_%s.pkl' % direction)
        
        avgtype = 'avg'
        if direction == 'EMF1':
            vars_obj = divars.EMF1()
        elif direction == 'EMF2':
            vars_obj = divars.EMF2()
        elif direction == 'EMF3':
            vars_obj = divars.EMF3()
        elif direction == 'Turb_Ekin':
            vars_obj = divars.TurbKineticEnergy()
        elif direction == 'KinHelicityRMS':
            vars_obj = divars.KinHelicity(GM=0.7692307692307692, omega=1.0)
            avgtype = 'rms'
        elif direction in ['vel1rms', 'vel2rms']:
            vars_obj = divars.Default(direction[:4])
            avgtype = 'rms'
        elif direction == 'vel3rms':
            vars_obj = divars.Vel3Fluct(GM=0.7692307692307692, omega=1.0)
            avgtype = 'rms'
        
        # reload if exists
        if os.path.isfile(outfile):
            with open(outfile, 'rb') as f:
                ops_obj = pkl.load(f)
        else:
            ops_obj = diops.ButterflyDiagram(vars_obj, \
                radii=[0.08, 0.11, 0.15, 0.2, 0.25], \
                x2min=(0.5*np.pi - dx2), x2max=(0.5*np.pi + dx2), \
                nz=1024, type=avgtype)
        
        ops_obj.read(athdf_files, verbose=True, \
                temp_save_path=outfile)
        
        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump(ops_obj, f)

In [5]:
if 'patternSpeed' in tasks['2D']:
    
    for direction in ['Bcc1','rho']:
        
        print('Processing direction %s' % direction)
        
        outfile = data_pathstem + ('patternSpeed_%s.pkl' % direction)
        
        vars_obj = divars.Default(direction)
        
        # reload if exists
        if os.path.isfile(outfile):
            with open(outfile, 'rb') as f:
                ops_obj = pkl.load(f)
        else:
            ops_obj = diops.PatternSpeed(vars_obj, \
                radii=[0.08, 0.11, 0.15, 0.2, 0.25])
        
        ops_obj.read(athdf_files, verbose=True, \
                temp_save_path=outfile)
        
        # save for reuse
        with open(outfile, 'wb') as f:
            pkl.dump(ops_obj, f)

Processing direction Bcc1
 - reading mhdLoops_strat.out2.00610.athdf.. done.
 - reading mhdLoops_strat.out2.00611.athdf.. done.
 - reading mhdLoops_strat.out2.00612.athdf.. done.
 - reading mhdLoops_strat.out2.00613.athdf.. done.
 - reading mhdLoops_strat.out2.00614.athdf.. done.
 - reading mhdLoops_strat.out2.00615.athdf.. done.
 - reading mhdLoops_strat.out2.00616.athdf.. done.
 - reading mhdLoops_strat.out2.00617.athdf.. done.
 - reading mhdLoops_strat.out2.00618.athdf.. done.
 - reading mhdLoops_strat.out2.00619.athdf.. done.
 - reading mhdLoops_strat.out2.00620.athdf.. done.
 - reading mhdLoops_strat.out2.00621.athdf.. done.
 - reading mhdLoops_strat.out2.00622.athdf.. done.
 - reading mhdLoops_strat.out2.00623.athdf.. done.
 - reading mhdLoops_strat.out2.00624.athdf.. done.
 - reading mhdLoops_strat.out2.00625.athdf.. done.
 - reading mhdLoops_strat.out2.00626.athdf.. done.
 - reading mhdLoops_strat.out2.00627.athdf.. done.
 - reading mhdLoops_strat.out2.00628.athdf.. done.
 - re

 - reading mhdLoops_strat.out2.00929.athdf.. done.
 - reading mhdLoops_strat.out2.00930.athdf.. done.
 - reading mhdLoops_strat.out2.00931.athdf.. done.
 - reading mhdLoops_strat.out2.00932.athdf.. done.
 - reading mhdLoops_strat.out2.00933.athdf.. done.
 - reading mhdLoops_strat.out2.00934.athdf.. done.
 - reading mhdLoops_strat.out2.00935.athdf.. done.
 - reading mhdLoops_strat.out2.00936.athdf.. done.
 - reading mhdLoops_strat.out2.00937.athdf.. done.
 - reading mhdLoops_strat.out2.00938.athdf.. done.
 - reading mhdLoops_strat.out2.00939.athdf.. done.
 - reading mhdLoops_strat.out2.00940.athdf.. done.
 - reading mhdLoops_strat.out2.00941.athdf.. done.
 - reading mhdLoops_strat.out2.00942.athdf.. done.
 - reading mhdLoops_strat.out2.00943.athdf.. done.
 - reading mhdLoops_strat.out2.00944.athdf.. done.
 - reading mhdLoops_strat.out2.00945.athdf.. done.
 - reading mhdLoops_strat.out2.00946.athdf.. done.
 - reading mhdLoops_strat.out2.00947.athdf.. done.
 - reading mhdLoops_strat.out2.

 - reading mhdLoops_strat.out2.00902.athdf.. done.
 - reading mhdLoops_strat.out2.00903.athdf.. done.
 - reading mhdLoops_strat.out2.00904.athdf.. done.
 - reading mhdLoops_strat.out2.00905.athdf.. done.
 - reading mhdLoops_strat.out2.00906.athdf.. done.
 - reading mhdLoops_strat.out2.00907.athdf.. done.
 - reading mhdLoops_strat.out2.00908.athdf.. done.
 - reading mhdLoops_strat.out2.00909.athdf.. done.
 - reading mhdLoops_strat.out2.00910.athdf.. done.
 - reading mhdLoops_strat.out2.00911.athdf.. done.
 - reading mhdLoops_strat.out2.00912.athdf.. done.
 - reading mhdLoops_strat.out2.00913.athdf.. done.
 - reading mhdLoops_strat.out2.00914.athdf.. done.
 - reading mhdLoops_strat.out2.00915.athdf.. done.
 - reading mhdLoops_strat.out2.00916.athdf.. done.
 - reading mhdLoops_strat.out2.00917.athdf.. done.
 - reading mhdLoops_strat.out2.00918.athdf.. done.
 - reading mhdLoops_strat.out2.00919.athdf.. done.
 - reading mhdLoops_strat.out2.00920.athdf.. done.
 - reading mhdLoops_strat.out2.